In [1]:
!pip install langgraph langchain langchain-google-genai google-generativeai

In [2]:
from langchain.tools import tool
import requests
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
@tool
def get_weather(city: str) -> str:
    """Get current weather for a given city using WeatherAPI.com."""
    api_key = os.getenv("WEATHER_API_KEY")
    if not api_key:
        return "WeatherAPI key not set. Please set WEATHER_API_KEY environment variable."

    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"
    response = requests.get(url)
    return response.json()
    # if response.status_code != 200:
    #     return f"Failed to fetch weather data: {response.json().get('error', {}).get('message', 'Unknown error')}"

    # data = response.json()
    # location = data["location"]["name"]
    # country = data["location"]["country"]
    # temp_c = data["current"]["temp_c"]
    # condition = data["current"]["condition"]["text"]
    # wind_kph = data["current"]["wind_kph"]

    # return f"The weather in {location}, {country} is {condition} with {temp_c}°C temperature and wind speed of {wind_kph} kph."

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=os.getenv("GOOGLE_API_KEY"))
tools = [get_weather]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example run
print(agent.run("What is the weather in Noida, Uttar Pradesh right now?"))

/Users/yash/Library/ai-travel-planner/.travelai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/v3/hj2d2g150gld7k2cm6q9d9gh0000gn/T/ipykernel_21363/2058371853.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>



> Entering new AgentExecutor chain...
I need to find the current weather conditions in Noida, Uttar Pradesh. I can use the get_weather tool to get this information.
Action: get_weather
Action Input: Noida, Uttar Pradesh
Observation: {'location': {'name': 'Noida', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 28.57, 'lon': 77.32, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752208807, 'localtime': '2025-07-11 10:10'}, 'current': {'last_updated_epoch': 1752208200, 'last_updated': '2025-07-11 10:00', 'temp_c': 30.3, 'temp_f': 86.5, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 175, 'wind_dir': 'S', 'pressure_mb': 1003.0, 'pressure_in': 29.62, 'precip_mm': 0.23, 'precip_in': 0.01, 'humidity': 75, 'cloud': 75, 'feelslike_c': 33.5, 'feelslike_f': 92.3, 'windchill_c': 31.4, 'windchill_f': 88.5, 'heatindex_c': 35.7, 'heatindex_f': 96.2, 'dewpoint_c': 22.4, 'dewpoint_

In [5]:
print(agent.run("What is the weather in pilkhuwa, Uttar Pradesh right now?"))



> Entering new AgentExecutor chain...
I need to find out the current weather conditions in Pilkhuwa, Uttar Pradesh. I have access to a tool called `get_weather` which should provide this information.
Action: get_weather
Action Input: pilkhuwa, Uttar Pradesh
Observation: {'location': {'name': 'Pilkhuwa', 'region': 'Uttar Pradesh', 'country': 'India', 'lat': 28.7167, 'lon': 77.65, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1752208815, 'localtime': '2025-07-11 10:10'}, 'current': {'last_updated_epoch': 1752208200, 'last_updated': '2025-07-11 10:00', 'temp_c': 30.3, 'temp_f': 86.5, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 6.0, 'wind_kph': 9.7, 'wind_degree': 175, 'wind_dir': 'S', 'pressure_mb': 1003.0, 'pressure_in': 29.62, 'precip_mm': 0.39, 'precip_in': 0.02, 'humidity': 75, 'cloud': 75, 'feelslike_c': 33.8, 'feelslike_f': 92.9, 'windchill_c': 31.4, 'windchill_f': 88.6, 'heatindex_c': 36.2, 'heatind

In [6]:
import requests

class WeatherForecastTool:
    def __init__(self, api_key:str):
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5"

    def get_current_weather(self, place:str):
        """Get current weather of a place"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                "q": place,
                "appid": self.api_key,
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            raise e
    
    def get_forecast_weather(self, place:str):
        """Get weather forecast of a place"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                "q": place,
                "appid": self.api_key,
                "cnt": 40,
                "units": "metric"
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            raise e

In [7]:
import os
# from utils.weather_info import WeatherForecastTool
from langchain.tools import tool
from typing import List
from dotenv import load_dotenv

class WeatherInfoTool:
    def __init__(self):
        load_dotenv()
        self.api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
        self.weather_service = WeatherForecastTool(self.api_key)
        self.weather_tool_list = self._setup_tools()
    
    def _setup_tools(self) -> List:
        """Setup all tools for the weather forecast tool"""
        @tool
        def get_current_weather(city: str) -> str:
            """Get current weather for a city"""
            weather_data = self.weather_service.get_current_weather(city)
            if weather_data:
                temp = weather_data.get('main', {}).get('temp', 'N/A')
                desc = weather_data.get('weather', [{}])[0].get('description', 'N/A')
                return f"Current weather in {city}: {temp}°C, {desc}"
            return f"Could not fetch weather for {city}"
        
        @tool
        def get_weather_forecast(city: str) -> str:
            """Get weather forecast for a city"""
            forecast_data = self.weather_service.get_forecast_weather(city)
            if forecast_data and 'list' in forecast_data:
                forecast_summary = []
                for i in range(len(forecast_data['list'])):
                    item = forecast_data['list'][i]
                    date = item['dt_txt'].split(' ')[0]
                    temp = item['main']['temp']
                    desc = item['weather'][0]['description']
                    forecast_summary.append(f"{date}: {temp} degree celcius , {desc}")
                return f"Weather forecast for {city}:\n" + "\n".join(forecast_summary)
            return f"Could not fetch forecast for {city}"
    
        return [get_current_weather, get_weather_forecast]

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

tools = WeatherInfoTool().weather_tool_list
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=os.getenv("GOOGLE_API_KEY"))

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("What's the weather forecast for Delhi?")




> Entering new AgentExecutor chain...
I need to get the weather forecast for Delhi.
Action: get_weather_forecast
Action Input: Delhi
Observation: Weather forecast for Delhi:
2025-07-11: 33.9 degree celcius , light rain
2025-07-11: 33.99 degree celcius , light rain
2025-07-11: 34.12 degree celcius , overcast clouds
2025-07-11: 32.99 degree celcius , light rain
2025-07-11: 31.94 degree celcius , light rain
2025-07-11: 30.6 degree celcius , light rain
2025-07-12: 30.76 degree celcius , light rain
2025-07-12: 31.08 degree celcius , light rain
2025-07-12: 34.84 degree celcius , light rain
2025-07-12: 36.77 degree celcius , overcast clouds
2025-07-12: 36.27 degree celcius , overcast clouds
2025-07-12: 33.95 degree celcius , scattered clouds
2025-07-12: 32.57 degree celcius , broken clouds
2025-07-12: 32.57 degree celcius , overcast clouds
2025-07-13: 31.96 degree celcius , broken clouds
2025-07-13: 33.71 degree celcius , scattered clouds
2025-07-13: 37.2 degree celcius , few clouds
2025-07

In [9]:
print(response)

The weather forecast for Delhi from 2025-07-11 to 2025-07-16 includes a mix of light rain, overcast clouds, scattered clouds, broken clouds, and few clouds. Temperatures will range from approximately 26.88 to 39.39 degrees Celsius.


In [10]:
response = agent.run("What's the weather forecast for Noida?")



> Entering new AgentExecutor chain...
I need to get the weather forecast for Noida.
Action: get_weather_forecast
Action Input: Noida
Observation: Weather forecast for Noida:
2025-07-11: 32.51 degree celcius , light rain
2025-07-11: 32.7 degree celcius , overcast clouds
2025-07-11: 33.03 degree celcius , overcast clouds
2025-07-11: 32.05 degree celcius , light rain
2025-07-11: 31.2 degree celcius , light rain
2025-07-11: 30.46 degree celcius , light rain
2025-07-12: 30.31 degree celcius , scattered clouds
2025-07-12: 31.63 degree celcius , broken clouds
2025-07-12: 35.58 degree celcius , light rain
2025-07-12: 37.63 degree celcius , overcast clouds
2025-07-12: 36.82 degree celcius , broken clouds
2025-07-12: 34.65 degree celcius , few clouds
2025-07-12: 33.28 degree celcius , broken clouds
2025-07-12: 32.67 degree celcius , overcast clouds
2025-07-13: 32.02 degree celcius , broken clouds
2025-07-13: 33.92 degree celcius , scattered clouds
2025-07-13: 37.39 degree celcius , few clouds


In [11]:
print(response)

The weather forecast for Noida includes a mix of light rain, overcast clouds, scattered clouds, broken clouds, and few clouds with temperatures ranging from 26.42 to 39.04 degrees Celsius between 2025-07-11 and 2025-07-16.


In [12]:
from collections import defaultdict
from langchain.tools import tool
from typing import List
import os
from dotenv import load_dotenv
from datetime import datetime

class WeatherInfoTool:
    def __init__(self):
        load_dotenv()
        self.api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
        self.weather_service = WeatherForecastTool(self.api_key)
        self.weather_tool_list = self._setup_tools()
    
    def _setup_tools(self) -> List:
        @tool
        def get_current_weather(city: str) -> str:
            """Get current weather for a city"""
            weather_data = self.weather_service.get_current_weather(city)
            if weather_data:
                temp = weather_data.get('main', {}).get('temp', 'N/A')
                desc = weather_data.get('weather', [{}])[0].get('description', 'N/A')
                return f"Current weather in {city}: {temp}°C, {desc}"
            return f"Could not fetch weather for {city}"
        
        @tool
        def get_weather_forecast(city: str) -> str:
            """Get 5-day forecast summary (daily) for a city"""
            forecast_data = self.weather_service.get_forecast_weather(city)
            if forecast_data and 'list' in forecast_data:
                daily = defaultdict(list)
                for item in forecast_data['list']:
                    date = item['dt_txt'].split(' ')[0]
                    temp = item['main']['temp']
                    desc = item['weather'][0]['description']
                    daily[date].append((temp, desc))

                summary = []
                for date, entries in daily.items():
                    temps = [t for t, _ in entries]
                    descriptions = [d for _, d in entries]
                    common_desc = max(set(descriptions), key=descriptions.count)
                    min_temp = round(min(temps), 1)
                    max_temp = round(max(temps), 1)
                    emoji = self._emoji_for_description(common_desc)

                    summary.append(
                        f"📅 {date}: {emoji} {common_desc.title()}, {min_temp}°C – {max_temp}°C"
                    )
                return f"Weather forecast for {city.title()}:\n\n" + "\n".join(summary)
            return f"Could not fetch forecast for {city}"

        return [get_current_weather, get_weather_forecast]

    def _emoji_for_description(self, desc: str) -> str:
        """Map basic weather descriptions to emojis"""
        desc = desc.lower()
        if "rain" in desc: return "🌧️"
        if "cloud" in desc: return "☁️"
        if "clear" in desc: return "☀️"
        if "snow" in desc: return "❄️"
        if "thunder" in desc: return "⛈️"
        return "🌤️"


In [13]:
from langchain.agents import initialize_agent, AgentType

tools = WeatherInfoTool().weather_tool_list
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=os.getenv("GOOGLE_API_KEY"))

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("What's the weather forecast for Pilkhuwa?")




> Entering new AgentExecutor chain...
I need to get the weather forecast for Pilkhuwa.
Action: get_weather_forecast
Action Input: Pilkhuwa
Observation: Weather forecast for Pilkhuwa:

📅 2025-07-11: 🌧️ Light Rain, 27.3°C – 31.7°C
📅 2025-07-12: ☁️ Overcast Clouds, 27.3°C – 35.8°C
📅 2025-07-13: 🌧️ Light Rain, 28.6°C – 37.0°C
📅 2025-07-14: 🌧️ Light Rain, 25.9°C – 30.9°C
📅 2025-07-15: 🌧️ Light Rain, 25.1°C – 29.8°C
📅 2025-07-16: 🌧️ Light Rain, 24.9°C – 25.5°C
Thought:I have the weather forecast for Pilkhuwa for the next 5 days.
Final Answer: The weather forecast for Pilkhuwa is: 2025-07-11: Light Rain, 27.3°C – 31.7°C; 2025-07-12: Overcast Clouds, 27.3°C – 35.8°C; 2025-07-13: Light Rain, 28.6°C – 37.0°C; 2025-07-14: Light Rain, 25.9°C – 30.9°C; 2025-07-15: Light Rain, 25.1°C – 29.8°C. 2025-07-16: Light Rain, 24.9°C – 25.5°C

> Finished chain.


In [14]:
print(response)

The weather forecast for Pilkhuwa is: 2025-07-11: Light Rain, 27.3°C – 31.7°C; 2025-07-12: Overcast Clouds, 27.3°C – 35.8°C; 2025-07-13: Light Rain, 28.6°C – 37.0°C; 2025-07-14: Light Rain, 25.9°C – 30.9°C; 2025-07-15: Light Rain, 25.1°C – 29.8°C. 2025-07-16: Light Rain, 24.9°C – 25.5°C


In [16]:
ls = response.split(";")
for i in range(len(ls)):
    print(ls[i])

The weather forecast for Pilkhuwa is: 2025-07-11: Light Rain, 27.3°C – 31.7°C
 2025-07-12: Overcast Clouds, 27.3°C – 35.8°C
 2025-07-13: Light Rain, 28.6°C – 37.0°C
 2025-07-14: Light Rain, 25.9°C – 30.9°C
 2025-07-15: Light Rain, 25.1°C – 29.8°C. 2025-07-16: Light Rain, 24.9°C – 25.5°C
